# Explore GEMMA and comparison with BulkLMM.jl

In this interactive notebook, we will explore the usage of GEMMA package, the methods behind the implementation, its performance running on the BXD data, as compared with the BulkLMM.jl package in Julia. 


This is still a work in progress, as here only the use of GEMMA for association tests with **univariate linear mixed models** will be talked about...

In [18]:
pwd()

"/home/xyu/github/BulkLMM.jl/test"

In [2]:
cd("..")

In [3]:
include("BXDdata_for_test.jl");

In [4]:
(n, m, p)

(79, 35556, 7321)

In [5]:
size(pheno)

(79, 35556)

In [6]:
size(geno)

(79, 7321)

In [7]:
size(kinship)

(79, 79)

## Generate BIMBAM files for GEMMA

In [8]:
traitID = 7919;

In [19]:
# Will be using BXD data
## Include the script to generate data
include("../../src/readData.jl");

## Read in BXD data:
pheno_file = "../../data/bxdData/BXDtraits.csv"
pheno = readBXDpheno(pheno_file);
geno_file = "../../data/bxdData/BXDgeno_prob.csv"
geno = readGenoProb_ExcludeComplements(geno_file);


## Read in BXD data to the BIMBAM format:
transform_bxd_geno_to_gemma(geno_file, "BXDgeno_prob_bimbam.txt");
transform_bxd_pheno_to_gemma(pheno_file, "BXDpheno_bimbam.txt", traitID);

LoadError: SystemError: opening file "/home/xyu/github/src/readData.jl": No such file or directory

## How to... 

### Cite GEMMA:

- Software tool and univariate linear mixed models
Xiang Zhou and Matthew Stephens (2012). Genome-wide efficient mixed-model analysis for association studies. Nature Genetics. 44: 821-824.

- Multivariate linear mixed models
Xiang Zhou and Matthew Stephens (2014). Efficient multivariate linear mixed model algo- rithms for genome-wide association studies. Nature Methods. 11: 407-409.

- Bayesian sparse linear mixed models
Xiang Zhou, Peter Carbonetto and Matthew Stephens (2013). Polygenic modeling with Bayesian sparse linear mixed models. PLoS Genetics. 9(2): e1003264.

- Variance component estimation with individual-level or summary data
Xiang Zhou (2016). A unified framework for variance component estimation with summary statistics in genome-wide association studies. bioRxiv. 042846.

### Install and Setup GEMMA:

It is recommended to install precompiled binaries of GEMMA and use on a linux system.

1. Fetch the latest [stable release](https://github.com/genetics-statistics/GEMMA/releases) and download the .gz file.

2. Inside the directory where the .gz file is located, run `gunzip gemma.linux.gz` to unpack the file.

3. Make sure it is executable with `chomod u+x gemma-linux` and then execute with `gemma-linux`. 

4. Run GEMMA by first typing `alias gemma=path to gemma executable` and then it will be envoked by simply typing `gemma` in terminal.

Run `alias gemma=path to gemma executable` so that the GEMMA program will start by just typing `gemma` in terminal.

Here inside this notebook, we did something similar to creating an alias shortcut by

In [10]:
pwd()

"/home/xyu/github/BulkLMM.jl/test"

In [13]:
gemma = "/home/xyu/software/GEMMA/gemma-0.98.5-linux-static-AMD64"

"/home/xyu/software/GEMMA/gemma-0.98.5-linux-static-AMD64"

and testing GEMMA set up by

In [14]:
run(`$gemma -h`)

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021

 type ./gemma -h [num] for detailed help
 options: 
  1: quick guide
  2: file I/O related
  3: SNP QC
  4: calculate relatedness matrix
  5: perform eigen decomposition
  6: perform variance component estimation
  7: fit a linear model
  8: fit a linear mixed model
  9: fit a multivariate linear mixed model
 10: fit a Bayesian sparse linear mixed model
 11: obtain predicted values
 12: calculate snp variance covariance
 13: note
 14: debug options

The GEMMA software is distributed under the GNU General Public v3
   -license    show license information
   see also http://www.xzlab.org/software.html, https://github.com/genetics-statistics


Process(`/home/xyu/software/GEMMA/gemma-0.98.5-linux-static-AMD64 -h`, ProcessExited(0))

## Input File Formats:

It is recommended to use [BIMBAM](https://stephenslab.uchicago.edu/software.html) (Bayesian Imputation Based Association Mapping) file format for imputed genotypes as well as for general covariates other than SNPs. BIMBAM format consists of three files, a mean genotype file, a phenotype file, and an optional SNP annotation file, which we will explain in detail below.

(We first run `gemma-generateData.jl` to generate BIMBAM format genotype data from origional. For he univariate trait data we picked the 7919-th column of the original BXD phenotype data)

In [20]:
# Create variables for shortcut references to the datafiles:
pheno_filename = "BXDpheno_bimbam.txt";
geno_filename = "BXDgeno_prob_bimbam.txt";

### Mean Genotype File

In [16]:
readdlm(geno_filename, ',')[1:6, :]

LoadError: ArgumentError: Cannot open 'BXDgeno_prob_bimbam.txt': not a file

In [17]:
size(readdlm(geno_filename, ','))

LoadError: ArgumentError: Cannot open 'BXDgeno_prob_bimbam.txt': not a file

The mean genotype file is like the above, where the first column is SNP id, the second and third column are allele types with minor allele first, and the remaining columns are the posterior/imputed mean genotypes of different individuals numbered from 0 and 2.

GEMMA codes alleles exactly as provided in the mean genotype file, and ignores the allele types given in the second and third columns. Therefore, the minor allele is the effect allele only if it is coded as 1 and the major allele is coded as 0. Missing genotypes are represented as "NA" values.

In our BXD data examples, we have two SNPs, "A" being the minor allele and "B" being the major allele, and observations at 7321 marker places for the 79 individuals. Therefore, as shown in the data, each row are the observations of frequencies of the minor allele "A" for the 79 individuals at a certain marker place, and each column are the observations of the "A" frequencies at the 7321 marker places for a certain individual.

### Phenotype File

In [244]:
readdlm(pheno_filename, ',')[:]

79-element Vector{Float64}:
 7.171
 7.115
 6.902
 7.192
 7.057
 6.893
 6.795
 6.889
 7.069
 7.048
 6.85
 7.116
 7.083
 ⋮
 7.034
 6.864
 6.839
 6.951
 7.049
 6.869
 6.912
 7.042
 7.224
 7.024
 7.048
 7.132

In [245]:
size(readdlm(pheno_filename, ','))

(79, 1)

For univariate trait BIMBAM data, it just one column of phenotype values of the 79 strains.

For binary traits, it is recommended to label controls as 0 and cases as 1 for better interpretation of the results.

### Relatedness Matrix File Format

### Original Matrix Format

Two formats of kinship matrix in the original matrix form allowed by GEMMA are as shown below:

In [246]:
example1_K = reshape([0.3345  -0.0227   0.0103 -0.0227  0.3032  -0.0253 0.0103  -0.0253   0.3531], 3, 3)

3×3 Matrix{Float64}:
  0.3345  -0.0227   0.0103
 -0.0227   0.3032  -0.0253
  0.0103  -0.0253   0.3531

In [247]:
example2_K = df = DataFrame(column_1 = ["id1", "id1", "id1", "id2", "id2", "id3"], 
               column_2 = ["id1", "id2", "id3", "id2", "id3", "id3"],
               column_3 = [0.3345, -0.0227, 0.0103, 0.3032,  -0.0253, 0.3531])

Row,column_1,column_2,column_3
,String,String,Float64
1,id1,id1,0.3345
2,id1,id2,-0.0227
3,id1,id3,0.0103
4,id2,id2,0.3032
5,id2,id3,-0.0253
6,id3,id3,0.3531


Two example kinship matrix formats allowed are as shown above, for three individuals. One can use `-km [num]` to choose which format to use, i.e. use `km -1` to accompany BIMBAM format.

## Models

### Estimate Relatedness Matrix from Genotypes

#### Basic Usage

`gemma -g [genotype file name] -p [phenotype file name] -gk [num: 1 for G_centered, 2 for G_standardized] -o [output prefix]`

In [24]:
@time run(`$gemma -g $geno_filename -p $pheno_filename -gk 1 -o kinship`)

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 79
## number of analyzed individuals = 79
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Calculating Relatedness Matrix ... 
================================================== 100%


**** INFO: Done.


  0.477868 seconds (645 allocations: 113.695 KiB)


Process(`/home/xyu/software/GEMMA/gemma-0.98.5-linux-static-AMD64 -g BXDgeno_prob_bimbam.txt -p BXDpheno_bimbam.txt -gk 1 -o kinship`, ProcessExited(0))

#### Detailed Information

$$G_c = \frac{1}{p}\sum_{i=1}^p(x_i -1_n \bar x_i)(x_i - 1_n \bar x_i)^T$$

$$G_s = \frac{1}{p}\sum_{i=1}^p\frac{1}{v_{x_i}}(x_i -1_n \bar x_i)(x_i - 1_n \bar x_i)^T$$

Which of the two relatedness matrix to choose will largely depend on the underlying genetic architecture of the given trait. Specifically, if SNPs with lower minor allele frequency tend to have larger effects (which is inversely proportional to its genotype variance), then the standardized genotype matrix is preferred. If the SNP effect size does not depend on its minor allele frequency, then the centered genotype matrix is preferred.

### Univariate Linear Mixed Models

#### Basic Usage

`gemma -g [filename] -p [filename] -a [(optional) SNP annotation] -k [relatedness matrix file name] -lmm [1 for Wald test, 2 for LR test, 3 for score test, 4 for all three tests] -o [prefix]`

In [268]:
pwd()

"/home/zyu20/git/BulkLMM.jl/test/run-gemma"

In [35]:
geno_filename

"BXDgeno_prob_bimbam.txt"

In [269]:
@time run(`$gemma -g $geno_filename -p $pheno_filename -k output/kinship.cXX.txt -lmm 2 -lmax 1000000 -o gemma_results.txt`)

GEMMA 0.98.5 (2021-08-25) by Xiang Zhou, Pjotr Prins and team (C) 2012-2021
Reading Files ... 
## number of total individuals = 79
## number of analyzed individuals = 79
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     7321
## number of analyzed SNPs         =     7321
Start Eigen-Decomposition...
pve estimate =0.813567
se(pve) =0.0950615
================================================== 100%
  1.036051 seconds (816 allocations: 63.141 KiB)


**** INFO: Done.


Process(`/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64 -g BXDgeno_prob_bimbam.txt -p BXDpheno_bimbam.txt -k output/kinship.cXX.txt -lmm 2 -lmax 1000000 -o gemma_results.txt`, ProcessExited(0))

#### Output Files:

There will be two optput files, both inside a folder named 'output' (auto-created) in the current directory:

- The 'pre-fix.log.txt' file contains some detailed information about the running parameters and computation time,

In [270]:
logreport_gemma = CSV.read("output/gemma_results.txt.log.txt", DataFrame, delim = '\t', header = false);

In [29]:
results_gemma = CSV.read("output/gemma_results.txt.assoc.txt", DataFrame, delim = '\t', header = false);

Full results reported by GEMMA has the format as shown above:

In [272]:
unique(results_gemma[:, 1])

2-element Vector{String3}:
 "chr"
 "-9"

#### Using the kinship matrix $G_c$ calculated by *GEMMA* using the above formula:

In [273]:
kinship_Gc = CSV.read("output/kinship.cXX.txt", DataFrame, delim = '\t', header = false);

In [274]:
kinshipMat_Gc = Matrix(kinship_Gc);

In [275]:
kinshipMat_Gc[1:6, 1:6]

6×6 Matrix{Float64}:
  0.971367    -0.0803409   0.118316    0.0516335  -0.00965022  -0.0889471
 -0.0803409    0.991482   -0.0287038   0.0275875  -0.141933    -0.0646924
  0.118316    -0.0287038   1.01951    -0.217635   -0.103393     0.116334
  0.0516335    0.0275875  -0.217635    1.04452     0.177017    -0.0793848
 -0.00965022  -0.141933   -0.103393    0.177017    0.996395    -0.0755602
 -0.0889471   -0.0646924   0.116334   -0.0793848  -0.0755602    0.943166

In [276]:
pheno_y = reshape(pheno[:, traitID], :, 1)

79×1 Matrix{Float64}:
 10.715
 10.792
 11.968
 11.56
 11.097
 10.85
 11.493
 10.76
 11.914
 10.37
 11.828
 10.611
 10.886
  ⋮
 11.912
 11.074
 12.058
 10.804
 10.526
 11.711
 10.516
 11.662
 12.082
 11.434
 10.81
 11.797

In [277]:
results_bulklmm_alt = scan(pheno_y, geno, kinshipMat_Gc; reml = false, method = "alt");

In [278]:
results_bulklmm_null = scan(pheno_y, geno, kinshipMat_Gc; reml = true, method = "null");

In [261]:
results_bulklmm_null[2] # hsq under the null model estimated by BulkLMM, using reml = true

0.28644367458715875

In [188]:
0.221055/(0.221055+0.0490897)

0.8182836827818573

#### Using the kinship matrix calculated by *BulkLMM.jl*:

In [189]:
kinshipMat_bulklmm = kinship;
kinshipMat_bulklmm[1:6, 1:6]

6×6 Matrix{Float64}:
 1.0       0.468763  0.561085  0.521296  0.502617  0.476443
 0.468763  1.0       0.482652  0.50435   0.431553  0.483647
 0.561085  0.482652  1.0       0.374733  0.443817  0.567154
 0.521296  0.50435   0.374733  1.0       0.577574  0.462847
 0.502617  0.431553  0.443817  0.577574  1.0       0.476722
 0.476443  0.483647  0.567154  0.462847  0.476722  1.0

In [190]:
results_bulklmm2_alt = scan(pheno_y, geno, kinshipMat_bulklmm; reml = false, method = "alt");

In [191]:
results_bulklmm2_null = scan(pheno_y, geno, kinshipMat_bulklmm; reml = true, method = "null");

In [192]:
results_bulklmm2_null[2] # hsq under the null model estimated by BulkLMM, using reml = true

5.973045472187468e-15

In [193]:
function lod2logp(LOD,df) 
    return -log10(1-cdf(Chisq(df),2*log(10)*LOD))
end

lod2logp (generic function with 1 method)

In [194]:
function p2lod(pval::Float64, df::Int64)
    
    lrs = invlogcdf(Chisq(df), log(1-pval))
    lod = lrs/(2*log(10))
    
    # return lrs
    return lod

end

p2lod (generic function with 1 method)

In [195]:
# load package
using Distributions

# LOD score
LOD = 3.0
# p-value from chi-square with 2df calculated from 
# cdf (cumulative distribution function)
p = 1-cdf(Chisq(2),2*log(10)*LOD)

0.0010000000000000009

When the LRT statistic is asymptotic Chisq(2) under the null, we would expect the corresponding LOD is equal to $-log_{10}(p)$

In [196]:
p2lod(p, 2)

2.999999999999999

In [197]:
-log10(p)

2.9999999999999996

In [198]:
using DelimitedFiles

In [31]:
results_gemma = readdlm("output/gemma_results.txt.assoc.txt", '\t')

7322×10 Matrix{Any}:
   "chr"  "rs"             "ps"  …     "logl_H1"   "l_mle"   "p_lrt"
 -9       "rs31443144"   -9         -48.1777      1.0e6     0.968104
 -9       "rs6269442"    -9         -48.1777      1.0e6     0.968104
 -9       "rs32285189"   -9         -48.1777      1.0e6     0.968104
 -9       "rs258367496"  -9         -48.1777      1.0e6     0.968104
 -9       "rs32430919"   -9      …  -48.1777      1.0e6     0.968104
 -9       "rs36251697"   -9         -48.1777      1.0e6     0.968104
 -9       "rs30658298"   -9         -48.1777      1.0e6     0.968104
 -9       "rs51852623"   -9         -48.1777      1.0e6     0.968104
 -9       "rs31879829"   -9         -48.1777      1.0e6     0.968104
 -9       "rs36742481"   -9      …  -48.1777      1.0e6     0.968104
 -9       "rs6365999"    -9         -48.1777      1.0e6     0.968104
 -9       "rs13470446"   -9         -48.1777      1.0e6     0.968111
  ⋮                              ⋱                          
 -9       "rs31466210

In [34]:
results_gemma[1:6, 5:10]

6×6 Matrix{Any}:
 "allele1"  "allele0"   "af"     "logl_H1"   "l_mle"   "p_lrt"
 "A"        "B"        0.557  -48.1777      1.0e6     0.968104
 "A"        "B"        0.557  -48.1777      1.0e6     0.968104
 "A"        "B"        0.557  -48.1777      1.0e6     0.968104
 "A"        "B"        0.557  -48.1777      1.0e6     0.968104
 "A"        "B"        0.557  -48.1777      1.0e6     0.968104

In [232]:
lods_gemma = map(x -> p2lod(x, 1), results_gemma[2:end, 10]);

In [233]:
hcat(lods_gemma, results_bulklmm_alt[3])

7321×2 Matrix{Float64}:
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.094309     0.377264
 0.0943064    0.377254
 0.0133797    0.105451
 ⋮            
 0.108847     0.00669749
 0.108847     0.00669749
 0.108847     0.00669749
 0.162331     0.00937725
 0.017045     0.00844277
 0.017045     0.00844277
 0.00230892   0.0101505
 0.00381478   0.00740475
 0.00364596   0.00768969
 0.00535974   0.00500525
 0.000872884  0.0144911
 0.000872884  0.0144911

In [234]:
hcat(lods_gemma, results_bulklmm_null[3])

7321×2 Matrix{Float64}:
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.094309     0.391942
 0.0943064    0.391933
 0.0133797    0.123007
 ⋮            
 0.108847     5.46372e-5
 0.108847     5.46372e-5
 0.108847     5.46372e-5
 0.162331     0.000546461
 0.017045     0.0240064
 0.017045     0.0240064
 0.00230892   0.0227789
 0.00381478   0.0181487
 0.00364596   0.0186486
 0.00535974   0.0137141
 0.000872884  0.0235871
 0.000872884  0.0235871

In [235]:
findall(isnan.(lods_gemma))

Int64[]

In [203]:
sumSqDiff(reshape(lods_gemma, :, 1), reshape(results_bulklmm_null[3], :, 1))

NaN